In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from operator import add
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from tkinter import * 

ModuleNotFoundError: No module named 'plotly'

In [ ]:
sc = SparkContext("local", "First App")
sqlCtx = SQLContext(sc)
df = sqlCtx.read.json("FAOdatabase.json",  multiLine=True)

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
data = df.rdd.map(lambda line: line[0]).collect()
country_list = sc.parallelize(data, 1).distinct().collect()

In [ ]:
data = df.rdd.map(lambda line: (line[0], [line[5]])).reduceByKey(lambda accum, n: accum + n).collect()

In [ ]:
data = df.select(*( df.columns[i] for i in [0,5] + [x for x in range(8,61)] )).rdd.map(lambda line: line[0:-1]).collect()

In [ ]:
dicc = {}
for year in range(1961,2014):
    dicc['Y'+ str(year)] = 'sum'

prod_years = df.agg(dicc).schema.names
prod_sum = df.agg(dicc).rdd.map(lambda line: line[0:54]).collect()[0]

Sum = []
for i,elt in enumerate(prod_years):
    Sum.append([elt[5:9],prod_sum[i]])

Sum_ordered = sorted(Sum, key=lambda tup: tup[0])

df2 = spark.createDataFrame(Sum_ordered)

x = [int(row['_1']) for row in  df2.select('_1').collect()]
y = [int(row['_2']) for row in df2.select('_2').collect()]

plt.plot(x,y)
plt.show()

In [ ]:
who = df.filter(df.Y1961==112227).rdd.map(lambda line: [line[0]]+[line[5]]).collect()[0]

In [ ]:
df_geo = spark \
    .read \
    .format('csv') \
    .options(header='true', inferSchema='true', delimiter=';') \
    .load('./Book1.csv')

df_inter = df.join(df_geo, df.Area == df_geo.Country)

In [ ]:
dicc = {}
for year in range(1961,2014):
    dicc['Y'+ str(year)] = 'sum'
    
prod_by_zones = {}
for year in range(1961,2014):
    prod_by_zones[str(year)] = []
    
for zone in ['Asia & Pacific', 'Europe', 'Arab States', 'Africa','South/Latin America', 'Unknown', 'North America']:
    
    prod_years = df_inter.filter(df_inter.Zone == zone).agg(dicc).schema.names
    prod_sum = df_inter.filter(df_inter.Zone == zone).agg(dicc).rdd.map(lambda line: line[0:54]).collect()[0]

    Sum = []
    for i,elt in enumerate(prod_years):
        Sum.append([elt[5:9],prod_sum[i]])

    Sum_ordered = sorted(Sum, key=lambda tup: tup[0])

    for year in Sum_ordered:
        prod_by_zones[year[0]] = prod_by_zones[year[0]] + [(zone,year[1])]

AP=[]
E=[]
AS=[]
A=[]
SA=[]
U=[]
NA=[]

#years = ['1961', '1962' ,'1963']
years = [str(x) for x in range(1961,2014)]

for year in years:
    AP.append(prod_by_zones[year][0][1])
    E.append(prod_by_zones[year][1][1])
    AS.append(prod_by_zones[year][2][1])
    A.append(prod_by_zones[year][3][1])
    SA.append(prod_by_zones[year][4][1])
    U.append(prod_by_zones[year][5][1])
    NA.append(prod_by_zones[year][6][1])
    
fig = go.Figure()
fig.add_trace(go.Bar(
    y=years,
    x=AP,
    name='Asia & Pacific',
    orientation='h',
    marker=dict(
        color='red',
        line=dict(color='black', width=0.25)
    )
))
fig.add_trace(go.Bar(
    y=years,
    x=E,
    name='Europe',
    orientation='h',
    marker=dict(
        color='blue',
        line=dict(color='black', width=0.25)
    )
))
fig.add_trace(go.Bar(
    y=years,
    x=AS,
    name='Arab States',
    orientation='h',
    marker=dict(
        color='green',
        line=dict(color='black', width=0.25)
    )
))
fig.add_trace(go.Bar(
    y=years,
    x=A,
    name='Africa',
    orientation='h',
    marker=dict(
        color='yellow',
        line=dict(color='black', width=0.25)
    )
))
fig.add_trace(go.Bar(
    y=years,
    x=SA,
    name='South/Latin America',
    orientation='h',
    marker=dict(
        color='orange',
        line=dict(color='black', width=0.25)
    )
))
fig.add_trace(go.Bar(
    y=years,
    x=NA,
    name='North America',
    orientation='h',
    marker=dict(
        color='magenta',
        line=dict(color='black', width=0.25)
    )
))

fig.update_layout(barmode='stack')
fig.show()

In [ ]:
dicc2 ={'Y2013':'sum'}
data = df.rdd.map(lambda line: line[0]).collect()
country_list = sc.parallelize(data, 1).distinct().collect()


In [ ]:
prod = {}
for country in country_list:
    prod[country] = df.filter(df.Area == country).agg(dicc2).rdd.map(lambda line: line[0]).collect()[0]

In [ ]:
for elt in list(prod.items()):
    if prod[elt[0]]== None:
        prod[elt[0]]=0
    
coor = df.select(*( df.columns[i] for i in [0,61,62] )).distinct().rdd.map(lambda line: (line[0], line[1], line[2])).collect()
df3 = spark.createDataFrame(coor)

lati = [int(row['_2']) for row in  df3.select('_2').collect()]
long = [int(row['_3']) for row in  df3.select('_3').collect()]
name = [row['_1'] for row in  df3.select('_1').collect()]

N = {}
for i,elt in enumerate(name):
    N[elt] = (lati[i],long[i])

N1 = list(N.items())
N2 = sorted(N1, key=lambda tup: tup[0]) 

produc = list(prod.values())

latitude = []
for elt in N2:
    latitude.append(elt[1][0])
    
longitude = []
for elt in N2:
    longitude.append(elt[1][1])

names = []
for elt in N2:
    names.append(elt[0])

fig = go.Figure(data=go.Scattergeo(
        lon = longitude,
        lat = latitude,
        text = names,
        mode = 'markers',
        marker = dict(
            size = 8,
            opacity = 0.8,
            reversescale = False,
            autocolorscale = False,
            symbol = 'square',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            colorscale = 'Blues',
            cmin = 0,
            color = produc,
            cmax =100000,
            colorbar_title="Food production (1000 tonnes)"
        )))

fig.update_layout(
        title = 'Countries 2013',
        geo = dict(
            scope='world',
            #projection_type='albers usa',
            showland = True,
            landcolor = "rgb(250, 250, 250)",
            subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(217, 217, 217)",
            countrywidth = 0.5,
            subunitwidth = 0.5
        ),
    )

fig.show()